# Pytorch 实现VAE(Variational Auto-Encoder,变分自编码器)
VAE是一种基于自编码器的深度**生成**模型

根据[博客](https://adaning.github.io/posts/53598.html)进行学习

# 理论知识
## AE(Auto-Encoder)
![AE(Auto-Encoder)](img/2022-10-11-18-01-58.png)

传统的自编码器可以分为Encoder和Decoder两个部分,但是实际上Encoder和Decoder并不能单独独立使用,因为中间层的Code并非是呈现一定的规律性.

## VAE
### 隐变量-概率分布式
理想形态下 生成模型可以被表述为$X=g(Z)$,但是由于没法直接知道$p(X)$,我们需要引入隐变量$Z$来求:
$$p(X) = \sum_Z p(X\mid Z) p(Z)$$
如果我们能把输入样本X编码得到的Z,即隐变量控制在某个分布中,我们就可以从隐变量的分布中采样,直接解码得到生成的结果,让Decoder独立工作,此时需要将隐变量建模为概率分布,而非是像AE一样看作是一个离散的值.
![VAE和AE在隐变量建模上的区别](img/2022-10-11-18-20-51.png)
每个样本都有一个自己专属的正态分布,样本之间必定存在重合,当采样到两个样本的叠加区域时,解码的内容会介于二者之间,例如满月到半月之间
![](img/2022-10-11-20-09-10.png)

每个分布的均值和方差需要通过神经网络直接拟合样本对应的正态分布$均值\mu和方差\sigma^2$
![](img/2022-10-11-20-10-53.png)
<!-- 在实际情况中,我们拟合的是$log \sigma^2$ -->

### KL散度-防止神经网络偷懒

在VAE任务中,一定要限制网络将方差学为0,这样就会把样本学成了一个点(离散值),导致VAE退化成AE.
我们使用KL散度来约束$p(Z|X)$,令其服从标准正态分布
> KL散度(KL Divergence,相对熵)用于衡量两个分布之间的差异性(信息损失),
> 假设P为真实的样本分布,Q为模型预测的分布,根据KL散度的公式formula:
> $$D_{\mathrm{KL}}(P \| Q)=\mathbb{E}_{\mathrm{x} \sim P}\left[\log \frac{P(x)}{Q(x)}\right]=\mathbb{E}_{\mathrm{x} \sim P}[\log P(x)-\log Q(x)]$$
> 当P,Q越接近时,$D_{\mathrm{KL}}$越小,当且仅当P和Q完全相同时,值为0
> KL散度还有两个性质
> 1. 非负
> 2. 不对称

求解过程如下:
$$
\begin{aligned}
&KL\Big(N(\mu,\sigma^2)\Big\Vert N(0,1)\Big)\\
=&\int \frac{1}{\sqrt{2\pi\sigma^2}}e^{-(x-\mu)^2/2\sigma^2} \left(\log \frac{e^{-(x-\mu)^2/2\sigma^2}/\sqrt{2\pi\sigma^2}}{e^{-x^2/2}/\sqrt{2\pi}}\right)dx\\\
=&\int \frac{1}{\sqrt{2\pi\sigma^2}}e^{-(x-\mu)^2/2\sigma^2} \log \left\{\frac{1}{\sqrt{\sigma^2}}\exp\left\{\frac{1}{2}\big[x^2-(x-\mu)^2/\sigma^2\big]\right\} \right\}dx\\\
=&\frac{1}{2}\int \frac{1}{\sqrt{2\pi\sigma^2}}e^{-(x-\mu)^2/2\sigma^2} \Big[-\log \sigma^2+x^2-(x-\mu)^2/\sigma^2 \Big] dx \\
=&\frac{1}{2}(-\log\sigma^2+\mu^2+\sigma^2-1)
\end{aligned}
$$
在求解时,需要Minimize Dkl
VAE常见的损失函数为:
$$
\begin{aligned}
\mathcal{L} = & \mathcal{L}_\mathrm{Recon} + \mathcal{L}_\mathrm{KL} \\
= & \mathcal{D}(\hat{X}_k,X_k)^2 + KL\Big(N(\mu,\sigma^2)\Big\Vert N(0,1)\Big)
\end{aligned}
$$
即重构损失和KL散度两部分


### 梯度断裂和重参数

我们想要用SGD或者其他优化方法来优化$p(Z|X_k)$的均值与方差,但是Sample这个操作并不可导,VAE利用**重参数化技巧**(Reparameterization Trick)使得梯度不因采样而断裂(无法进行).
![](img/2022-10-11-20-45-17.png)
因为Z的导数可以写成:
$$
\begin{aligned}&\frac{1}{\sqrt{2\pi\sigma^2}}\exp\left(-\frac{(z-\mu)^2}{2\sigma^2}\right)dz \\
=& \frac{1}{\sqrt{2\pi}}\exp\left[-\frac{1}{2}\left(\frac{z-\mu}{\sigma}\right)^2\right]d\left(\frac{z-\mu}{\sigma}\right)
\end{aligned}
$$
可以看出 $(z - \mu) / \sigma^2 \sim \mathcal{N}(0, I)$,从$\mathcal{N}(\mu, \sigma^2)$中采样,相当于从标准正态分布中采样出了一个噪声$\epsilon$,通过放缩$Z= \mu + \epsilon \times \sigma$即可恢复,这样断裂的锅甩给了噪声这个无关变量,使得均值和方差可以继续参与优化.

# 代码实现

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
import matplotlib.pyplot as plt

In [36]:
latent_dim = 2
input_dim = 28*28
inter_dim = 256


class VAE(nn.Module):
    def __init__(self, input_dim, inter_dim, latent_dim):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, inter_dim),
            nn.ReLU(),
            nn.Linear(inter_dim, latent_dim*2) # 2是因为mu和logvar各自是一个
        )

        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, inter_dim),
            nn.ReLU(),
            nn.Linear(inter_dim, input_dim),
            nn.Sigmoid()
        )
    def reparameter(self, mu, logvar):
        epsilon = torch.randn_like(mu)
        return mu+epsilon*torch.exp(logvar/2)

    def forward(self,x):
        org_size = x.size()
        batch = org_size[0]
        x = x.view(batch,-1) # flatten
        h = self.encoder(x)
        mu,logvar = h.chunk(2,dim =1) # chunk 按照dim将tensor分割成两个tensor
        z = self.reparameter(mu,logvar)
        recon_x = self.decoder(z).view(size=org_size)
        return recon_x, mu, logvar
    
vae=VAE(input_dim,inter_dim,latent_dim)


In [ ]:
## 网络参数量
from thop import profile 
flops, params = profile(vae.encoder, inputs=(torch.randn(784),))
print(flops)
print(params)

In [56]:
## 网络模型可视化
import netron
netron.start("model/vae.pth",address=8080)

Stopping http://localhost:8080
Serving 'model/vae.pth' at http://localhost:8080


('localhost', 8080)